In [15]:
import sys
# sys.path.append('/home/wgh24/Code/CartER-single')
print(sys.path)


['/home/wgh24/Code/CartER-single/notebooks', '/usr/lib64/python39.zip', '/usr/lib64/python3.9', '/usr/lib64/python3.9/lib-dynload', '', '/home/wgh24/Code/CartER-single/.venv/lib64/python3.9/site-packages', '/home/wgh24/Code/CartER-single/.venv/lib/python3.9/site-packages', '/home/wgh24/Code/CartER-single']


In [ ]:
import time

In [ ]:
# %load_ext autoreload
# %autoreload 2

from commander.network.network import NetworkManager
from commander.network.protocol import (
    PingPacket,
    SetPositionPacket,
    SetVelocityPacket,
    FindLimitsPacket,
    ObservationPacket,
    RequestDebugInfoPacket,
    CheckLimitPacket,
    DoJigglePacket,
    DebugPacket,
    InfoPacket,
    ErrorPacket,
    ExperimentStartPacket,
    ExperimentStopPacket,
    SetMaxVelocityPacket
)
from commander.network.network_constants import SetOperation, CartID
from commander.log import setup_logging

from serial.tools import list_ports

from time import time, sleep
import random
import numpy as np

setup_logging(file=False)

port = list_ports.comports()[0].device
print([port.device for port in list_ports.comports()])
print(port)

nm = NetworkManager(port=port)
nm.open()

nm_reset = NetworkManager(port="/dev/ttyACM0")
nm_reset.open()
nm_reset.close()

initialised = False
while not initialised:
    try:
        nm.read_initial_output()
    except UnicodeDecodeError:
        pass
    else:
        initialised = True

In [38]:
print(nm.serial.read_all().decode("ASCII", errors="ignore"))

In [39]:
nm.dump_packets()

# PingPacket

In [135]:
ping_pkt = PingPacket(timestamp=10)
nm.send_packet(ping_pkt)

pkt = nm.read_packet()
print(pkt)

commander.network.network     : DEBUG    Sent packet: <PingPacket: id_: b'p', read_time: None, timestamp: 10>
commander.network.network     : DEBUG    Sent packet: <PingPacket: id_: b'p', read_time: None, timestamp: 10>
commander.network.network     : DEBUG    Sent packet: <PingPacket: id_: b'p', read_time: None, timestamp: 10>
commander.network.network     : DEBUG    Sent packet: <PingPacket: id_: b'p', read_time: None, timestamp: 10>
commander.network.network     : DEBUG    Sent packet: <PingPacket: id_: b'p', read_time: None, timestamp: 10>
commander.network.network     : DEBUG    Sent packet: <PingPacket: id_: b'p', read_time: None, timestamp: 10>
commander.network.network     : DEBUG    Sent packet: <PingPacket: id_: b'p', read_time: None, timestamp: 10>
commander.network.network     : DEBUG    Sent packet: <PingPacket: id_: b'p', read_time: None, timestamp: 10>
commander.network.network     : DEBUG    Sent packet: <PingPacket: id_: b'p', read_time: None, timestamp: 10>
commander.

<PongPacket: id_: b'P', read_time: 2022-08-16 09:46:42.386625, timestamp: 10>


# UnknownPacket

In [134]:
nm.serial.write(b"a")
pkt = nm.read_packet()
print(pkt)

<DebugPacket: id_: b'#', msg: Received unknown packet with ID: 97, read_time: 2022-08-16 09:46:39.684379>


# RequestDebugInfoPacket

In [ ]:
request_dbg_pkt = RequestDebugInfoPacket()
nm.send_packet(request_dbg_pkt)

sleep(0.010)
nm.dump_packets()

# FindLimitsPacket

In [122]:
PKT = FindLimitsPacket
nm.dump_packets()

pkt = PKT()
nm.send_packet(pkt)
print(f"Sent packet: {pkt}")

response_pkt = nm.get_packet(PKT, block=True, callback=nm.printer_callback(excepts=(PKT, )))
nm.dump_packets()
print(response_pkt)

commander.network.network     : DEBUG    Sent packet: <FindLimitsPacket: id_: b'|', read_time: None>
commander.network.network     : DEBUG    Sent packet: <FindLimitsPacket: id_: b'|', read_time: None>
commander.network.network     : DEBUG    Sent packet: <FindLimitsPacket: id_: b'|', read_time: None>
commander.network.network     : DEBUG    Sent packet: <FindLimitsPacket: id_: b'|', read_time: None>
commander.network.network     : DEBUG    Sent packet: <FindLimitsPacket: id_: b'|', read_time: None>
commander.network.network     : DEBUG    Sent packet: <FindLimitsPacket: id_: b'|', read_time: None>
commander.network.network     : DEBUG    Sent packet: <FindLimitsPacket: id_: b'|', read_time: None>
commander.network.network     : DEBUG    Sent packet: <FindLimitsPacket: id_: b'|', read_time: None>
commander.network.network     : DEBUG    Sent packet: <FindLimitsPacket: id_: b'|', read_time: None>
commander.network.network     : DEBUG    Sent packet: <FindLimitsPacket: id_: b'|', read_ti

Sent packet: <FindLimitsPacket: id_: b'|', read_time: None>
<InfoPacket: id_: b'~', msg: LimitFinder: LEFT LIMIT HIT [fast], read_time: 2022-08-16 09:44:49.817362>
<InfoPacket: id_: b'~', msg: LimitFinder: LEFT LIMIT RETRACTED, read_time: 2022-08-16 09:44:50.045884>
<InfoPacket: id_: b'~', msg: LimitFinder: LEFT LIMIT HIT [slow], read_time: 2022-08-16 09:44:50.580325>
<InfoPacket: id_: b'~', msg: LimitFinder: LEFT LIMIT SET, read_time: 2022-08-16 09:44:50.585232>
<InfoPacket: id_: b'~', msg: LimitFinder: RIGHT LIMIT HIT [fast], read_time: 2022-08-16 09:45:02.921819>
<InfoPacket: id_: b'~', msg: LimitFinder: RIGHT LIMIT RETRACTED, read_time: 2022-08-16 09:45:03.125005>


commander.network.network     : DEBUG    Read packet: <ExperimentInfoPacket: cart_id: 0, id_: b':', read_time: 2022-08-16 09:45:03.595663, specifier: 3, value: 25028>
commander.network.network     : DEBUG    Read packet: <ExperimentInfoPacket: cart_id: 0, id_: b':', read_time: 2022-08-16 09:45:03.595663, specifier: 3, value: 25028>
commander.network.network     : DEBUG    Read packet: <ExperimentInfoPacket: cart_id: 0, id_: b':', read_time: 2022-08-16 09:45:03.595663, specifier: 3, value: 25028>
commander.network.network     : DEBUG    Read packet: <ExperimentInfoPacket: cart_id: 0, id_: b':', read_time: 2022-08-16 09:45:03.595663, specifier: 3, value: 25028>
commander.network.network     : DEBUG    Read packet: <ExperimentInfoPacket: cart_id: 0, id_: b':', read_time: 2022-08-16 09:45:03.595663, specifier: 3, value: 25028>
commander.network.network     : DEBUG    Read packet: <ExperimentInfoPacket: cart_id: 0, id_: b':', read_time: 2022-08-16 09:45:03.595663, specifier: 3, value: 25028

<InfoPacket: id_: b'~', msg: LimitFinder: RIGHT LIMIT HIT [slow], read_time: 2022-08-16 09:45:03.591761>
<ExperimentInfoPacket: cart_id: 0, id_: b':', read_time: 2022-08-16 09:45:03.595663, specifier: 3, value: 25028>


commander.network.network     : DEBUG    Read packet: <FindLimitsPacket: id_: b'|', read_time: 2022-08-16 09:45:04.837429>
commander.network.network     : DEBUG    Read packet: <FindLimitsPacket: id_: b'|', read_time: 2022-08-16 09:45:04.837429>
commander.network.network     : DEBUG    Read packet: <FindLimitsPacket: id_: b'|', read_time: 2022-08-16 09:45:04.837429>
commander.network.network     : DEBUG    Read packet: <FindLimitsPacket: id_: b'|', read_time: 2022-08-16 09:45:04.837429>
commander.network.network     : DEBUG    Read packet: <FindLimitsPacket: id_: b'|', read_time: 2022-08-16 09:45:04.837429>
commander.network.network     : DEBUG    Read packet: <FindLimitsPacket: id_: b'|', read_time: 2022-08-16 09:45:04.837429>
commander.network.network     : DEBUG    Read packet: <FindLimitsPacket: id_: b'|', read_time: 2022-08-16 09:45:04.837429>
commander.network.network     : DEBUG    Read packet: <FindLimitsPacket: id_: b'|', read_time: 2022-08-16 09:45:04.837429>
commander.networ

<InfoPacket: id_: b'~', msg: LimitFinder: NOW DONE, read_time: 2022-08-16 09:45:04.835053>
<FindLimitsPacket: id_: b'|', read_time: 2022-08-16 09:45:04.837429>


# CheckLimitPacket

In [136]:
PKT = CheckLimitPacket
nm.dump_packets()

pkt = PKT()
nm.send_packet(pkt)
print(f"Sent packet: {pkt}")

response_pkt = nm.get_packet(PKT, block=True, callback=nm.printer_callback(excepts=(PKT, )))
nm.dump_packets()
print(response_pkt)

commander.network.network     : DEBUG    Sent packet: <CheckLimitPacket: id_: b'/', read_time: None>
commander.network.network     : DEBUG    Sent packet: <CheckLimitPacket: id_: b'/', read_time: None>
commander.network.network     : DEBUG    Sent packet: <CheckLimitPacket: id_: b'/', read_time: None>
commander.network.network     : DEBUG    Sent packet: <CheckLimitPacket: id_: b'/', read_time: None>
commander.network.network     : DEBUG    Sent packet: <CheckLimitPacket: id_: b'/', read_time: None>
commander.network.network     : DEBUG    Sent packet: <CheckLimitPacket: id_: b'/', read_time: None>
commander.network.network     : DEBUG    Sent packet: <CheckLimitPacket: id_: b'/', read_time: None>
commander.network.network     : DEBUG    Sent packet: <CheckLimitPacket: id_: b'/', read_time: None>
commander.network.network     : DEBUG    Sent packet: <CheckLimitPacket: id_: b'/', read_time: None>
commander.network.network     : DEBUG    Sent packet: <CheckLimitPacket: id_: b'/', read_ti

Sent packet: <CheckLimitPacket: id_: b'/', read_time: None>
<InfoPacket: id_: b'~', msg: LimitChecker: LEFT SUPER FAST MARGIN HIT, read_time: 2022-08-16 09:46:51.285522>
<InfoPacket: id_: b'~', msg: LimitChecker: LEFT LIMIT HIT [fast], read_time: 2022-08-16 09:46:51.672411>
<InfoPacket: id_: b'~', msg: LimitChecker: LEFT LIMIT RETRACTED, read_time: 2022-08-16 09:46:51.870128>


commander.network.network     : DEBUG    Read packet: <ExperimentInfoPacket: cart_id: 1, id_: b':', read_time: 2022-08-16 09:46:52.318408, specifier: 1, value: 96>
commander.network.network     : DEBUG    Read packet: <ExperimentInfoPacket: cart_id: 1, id_: b':', read_time: 2022-08-16 09:46:52.318408, specifier: 1, value: 96>
commander.network.network     : DEBUG    Read packet: <ExperimentInfoPacket: cart_id: 1, id_: b':', read_time: 2022-08-16 09:46:52.318408, specifier: 1, value: 96>
commander.network.network     : DEBUG    Read packet: <ExperimentInfoPacket: cart_id: 1, id_: b':', read_time: 2022-08-16 09:46:52.318408, specifier: 1, value: 96>
commander.network.network     : DEBUG    Read packet: <ExperimentInfoPacket: cart_id: 1, id_: b':', read_time: 2022-08-16 09:46:52.318408, specifier: 1, value: 96>
commander.network.network     : DEBUG    Read packet: <ExperimentInfoPacket: cart_id: 1, id_: b':', read_time: 2022-08-16 09:46:52.318408, specifier: 1, value: 96>
commander.networ

<InfoPacket: id_: b'~', msg: LimitChecker: LEFT LIMIT HIT [slow], read_time: 2022-08-16 09:46:52.306462>
<InfoPacket: id_: b'~', msg: LimitChecker: LEFT LIMIT GET, read_time: 2022-08-16 09:46:52.312220>
<InfoPacket: id_: b'~', msg: LimitChecker: New limit was 96, read_time: 2022-08-16 09:46:52.315212>
<ExperimentInfoPacket: cart_id: 1, id_: b':', read_time: 2022-08-16 09:46:52.318408, specifier: 1, value: 96>


commander.network.network     : DEBUG    Read packet: <CheckLimitPacket: id_: b'/', read_time: 2022-08-16 09:46:53.551356>
commander.network.network     : DEBUG    Read packet: <CheckLimitPacket: id_: b'/', read_time: 2022-08-16 09:46:53.551356>
commander.network.network     : DEBUG    Read packet: <CheckLimitPacket: id_: b'/', read_time: 2022-08-16 09:46:53.551356>
commander.network.network     : DEBUG    Read packet: <CheckLimitPacket: id_: b'/', read_time: 2022-08-16 09:46:53.551356>
commander.network.network     : DEBUG    Read packet: <CheckLimitPacket: id_: b'/', read_time: 2022-08-16 09:46:53.551356>
commander.network.network     : DEBUG    Read packet: <CheckLimitPacket: id_: b'/', read_time: 2022-08-16 09:46:53.551356>
commander.network.network     : DEBUG    Read packet: <CheckLimitPacket: id_: b'/', read_time: 2022-08-16 09:46:53.551356>
commander.network.network     : DEBUG    Read packet: <CheckLimitPacket: id_: b'/', read_time: 2022-08-16 09:46:53.551356>
commander.networ

<InfoPacket: id_: b'~', msg: LimitChecker: NOW DONE, read_time: 2022-08-16 09:46:53.548855>
<CheckLimitPacket: id_: b'/', read_time: 2022-08-16 09:46:53.551356>


# DoJigglePacket

In [ ]:
PKT = DoJigglePacket
nm.dump_packets()

pkt = PKT()
nm.send_packet(pkt)
print(f"Sent packet: {pkt}")

response_pkt = nm.get_packet(PKT, block=True, callback=nm.printer_callback(excepts=(PKT, )))
nm.dump_packets()
print(response_pkt)

# SetPositionPacket

In [127]:
nm.dump_packets()

pkt = SetPositionPacket(operation=SetOperation.EQUAL, cart_id=CartID.ONE, value=2000, actobs_tracker=1)
nm.send_packet(pkt)

print(f"Sent packet: {pkt}")

nm.dump_packets()

commander.network.network     : DEBUG    Sent packet: <SetPositionPacket: actobs_tracker: 1, cart_id: 1, id_: b'x', operation: =, value: 2000>
commander.network.network     : DEBUG    Sent packet: <SetPositionPacket: actobs_tracker: 1, cart_id: 1, id_: b'x', operation: =, value: 2000>
commander.network.network     : DEBUG    Sent packet: <SetPositionPacket: actobs_tracker: 1, cart_id: 1, id_: b'x', operation: =, value: 2000>
commander.network.network     : DEBUG    Sent packet: <SetPositionPacket: actobs_tracker: 1, cart_id: 1, id_: b'x', operation: =, value: 2000>
commander.network.network     : DEBUG    Sent packet: <SetPositionPacket: actobs_tracker: 1, cart_id: 1, id_: b'x', operation: =, value: 2000>
commander.network.network     : DEBUG    Sent packet: <SetPositionPacket: actobs_tracker: 1, cart_id: 1, id_: b'x', operation: =, value: 2000>
commander.network.network     : DEBUG    Sent packet: <SetPositionPacket: actobs_tracker: 1, cart_id: 1, id_: b'x', operation: =, value: 2000>

Sent packet: <SetPositionPacket: actobs_tracker: 1, cart_id: 1, id_: b'x', operation: =, value: 2000>


In [84]:
nm.dump_packets()
value = 10000

pkt = SetVelocityPacket(operation=SetOperation.ADD, cart_id=CartID.ONE, value=value, actobs_tracker=100)
nm.send_packet(pkt)
sleep(1)

pkt = SetVelocityPacket(operation=SetOperation.ADD, cart_id=CartID.ONE, value=-2*value, actobs_tracker=100)
nm.send_packet(pkt)

sleep(0.5)
pkt = SetVelocityPacket(operation=SetOperation.ADD, cart_id=CartID.ONE, value=value, actobs_tracker=100)
nm.send_packet(pkt)
sleep(0.5)

nm.dump_packets()

<InfoPacket: id_: b'~', msg: Not in constant velocity mode - SetVelocity ignored., read_time: 2022-08-15 16:21:03.979236>
<InfoPacket: id_: b'~', msg: Not in constant velocity mode - SetVelocity ignored., read_time: 2022-08-15 16:21:03.979320>
<InfoPacket: id_: b'~', msg: Not in constant velocity mode - SetVelocity ignored., read_time: 2022-08-15 16:21:03.979361>


# SetVelocityPacket

In [295]:
max_velo_pkt = SetMaxVelocityPacket(SetOperation.EQUAL, cart_id=CartID.ONE, value=20_000, actobs_tracker=2)
nm.send_packet(max_velo_pkt)

commander.network.network     : DEBUG    Sent packet: <SetMaxVelocityPacket: actobs_tracker: 2, cart_id: 1, id_: b'w', operation: =, value: 20000>
commander.network.network     : DEBUG    Sent packet: <SetMaxVelocityPacket: actobs_tracker: 2, cart_id: 1, id_: b'w', operation: =, value: 20000>
commander.network.network     : DEBUG    Sent packet: <SetMaxVelocityPacket: actobs_tracker: 2, cart_id: 1, id_: b'w', operation: =, value: 20000>
commander.network.network     : DEBUG    Sent packet: <SetMaxVelocityPacket: actobs_tracker: 2, cart_id: 1, id_: b'w', operation: =, value: 20000>
commander.network.network     : DEBUG    Sent packet: <SetMaxVelocityPacket: actobs_tracker: 2, cart_id: 1, id_: b'w', operation: =, value: 20000>
commander.network.network     : DEBUG    Sent packet: <SetMaxVelocityPacket: actobs_tracker: 2, cart_id: 1, id_: b'w', operation: =, value: 20000>
commander.network.network     : DEBUG    Sent packet: <SetMaxVelocityPacket: actobs_tracker: 2, cart_id: 1, id_: b'w',

In [304]:
nm.dump_packets()
value = 12000
wait = 0.42

for i in range(10):
    pkt = SetVelocityPacket(operation=SetOperation.EQUAL, cart_id=CartID.ONE, value=value, actobs_tracker=255)
    nm.send_packet(pkt)
    sleep(wait+i/60)

    pkt = SetVelocityPacket(operation=SetOperation.EQUAL, cart_id=CartID.ONE, value=-value, actobs_tracker=100)
    nm.send_packet(pkt)
    sleep(wait+i/50)

pkt = SetVelocityPacket(operation=SetOperation.EQUAL, cart_id=CartID.ONE, value=0, actobs_tracker=100)
nm.send_packet(pkt)

print(f"Sent packet: {pkt}")
sleep(1)

nm.dump_packets()

Sent packet: <SetVelocityPacket: actobs_tracker: 100, cart_id: 1, id_: b'v', operation: =, value: 0>


In [300]:
nm.dump_packets()

pkt = SetVelocityPacket(operation=SetOperation.EQUAL, cart_id=CartID.ONE, value=0, actobs_tracker=100)
nm.send_packet(pkt)

print(f"Sent packet: {pkt}")
sleep(1)

nm.dump_packets()

Sent packet: <SetVelocityPacket: actobs_tracker: 100, cart_id: 1, id_: b'v', operation: =, value: 0>


In [299]:
nm.dump_packets()

pkt = SetVelocityPacket(operation=SetOperation.ADD, cart_id=CartID.ONE, value=-3000, actobs_tracker=100)
nm.send_packet(pkt)

print(f"Sent packet: {pkt}")
sleep(1)

nm.dump_packets()

Sent packet: <SetVelocityPacket: actobs_tracker: 100, cart_id: 1, id_: b'v', operation: +, value: -3000>


In [4]:
nm.close()